# This is a notebook to show how to update the autoencoder weight in to the pre trained big ldm model.

In [1]:
import argparse, os, sys, glob
sys.path.append(os.getcwd()+"/ldm")
from omegaconf import OmegaConf
from PIL import Image
from tqdm import tqdm
import numpy as np
import torch
from ldm.util import instantiate_from_config
from ldm.models.diffusion.ddim import DDIMSampler
from torch.utils.data import Dataset, DataLoader

/home/nicky/anaconda3/envs/ldm-inpaint/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config_ldm = OmegaConf.load("ldm/models/ldm/inpainting_big/config.yaml")
config_auto = OmegaConf.load("ldm//models/first_stage_models/vq-f4-noattn/config.yaml")

ldm_model_path = "last.ckpt"
auto_model_path = "auto.ckpt"


In [3]:
##load state dict of autoencoder
autoencoder_dict=torch.load(auto_model_path)['state_dict']

In [4]:
##create ldm model and load weight
model = instantiate_from_config(config_ldm.model)
model.load_state_dict(torch.load(ldm_model_path)['state_dict'],strict=False)



LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 387.25 M params.
Keeping EMAs of 418.
making attention of type 'none' with 512 in_channels
Working with z of shape (1, 3, 64, 64) = 12288 dimensions.
making attention of type 'none' with 512 in_channels
Using first stage also as cond stage.


<All keys matched successfully>

In [5]:
##get the weight as a dictinoary
cache_state_dict=model.state_dict()

In [6]:
##update the weight dictionary
for key in cache_state_dict:
    if 'first_stage_model' in key or 'cond_stage_model' in key :
        print(key)
        state_name=key.split('.')[1:]
        state='.'.join(state_name)
        
        cache_state_dict[key] = autoencoder_dict[state]

first_stage_model.encoder.conv_in.weight
first_stage_model.encoder.conv_in.bias
first_stage_model.encoder.down.0.block.0.norm1.weight
first_stage_model.encoder.down.0.block.0.norm1.bias
first_stage_model.encoder.down.0.block.0.conv1.weight
first_stage_model.encoder.down.0.block.0.conv1.bias
first_stage_model.encoder.down.0.block.0.norm2.weight
first_stage_model.encoder.down.0.block.0.norm2.bias
first_stage_model.encoder.down.0.block.0.conv2.weight
first_stage_model.encoder.down.0.block.0.conv2.bias
first_stage_model.encoder.down.0.block.1.norm1.weight
first_stage_model.encoder.down.0.block.1.norm1.bias
first_stage_model.encoder.down.0.block.1.conv1.weight
first_stage_model.encoder.down.0.block.1.conv1.bias
first_stage_model.encoder.down.0.block.1.norm2.weight
first_stage_model.encoder.down.0.block.1.norm2.bias
first_stage_model.encoder.down.0.block.1.conv2.weight
first_stage_model.encoder.down.0.block.1.conv2.bias
first_stage_model.encoder.down.0.downsample.conv.weight
first_stage_mode

In [7]:
## load a updated state_dict 
model.load_state_dict(cache_state_dict)


## save the updated model
torch.save(model.state_dict(), "updated_ldm.ckpt")